In [1]:
library(readr)
default_of_credit_card_clients <- read_delim("C:/Users/dheer/Desktop/UTD/4-Spring 2017/R for Data Scientists/Assignment/7/default of credit card clients.txt", "\t", escape_double = FALSE, trim_ws = TRUE)

Warning message:
"Missing column names filled in: 'X1' [1]"Warning message:
"Duplicated column names deduplicated: 'X1' => 'X1_1' [2]"Parsed with column specification:
cols(
  .default = col_integer()
)
See spec(...) for full column specifications.


In [2]:
credit_card <- default_of_credit_card_clients
credit_card$X1 <- NULL
colnames(credit_card)[1] <- c("X1")

In [3]:
credit_card$X2 <- as.factor(credit_card$X2)
credit_card$X3 <- as.factor(credit_card$X3)
credit_card$X4 <- as.factor(credit_card$X4)

In [4]:
levels(credit_card$X3) <- c("1","1","2","3","4","4","4")
levels(credit_card$X4) <- c("1","1","2","3")

In [5]:
dim(credit_card)

[1] 30000    24

In [6]:
summary(credit_card)

       X1          X2        X3        X4              X5       
 Min.   :  10000   1:11888   1:10599   1:13713   Min.   :21.00  
 1st Qu.:  50000   2:18112   2:14030   2:15964   1st Qu.:28.00  
 Median : 140000             3: 4917   3:  323   Median :34.00  
 Mean   : 167484             4:  454             Mean   :35.49  
 3rd Qu.: 240000                                 3rd Qu.:41.00  
 Max.   :1000000                                 Max.   :79.00  
       X6                X7                X8                X9         
 Min.   :-2.0000   Min.   :-2.0000   Min.   :-2.0000   Min.   :-2.0000  
 1st Qu.:-1.0000   1st Qu.:-1.0000   1st Qu.:-1.0000   1st Qu.:-1.0000  
 Median : 0.0000   Median : 0.0000   Median : 0.0000   Median : 0.0000  
 Mean   :-0.0167   Mean   :-0.1338   Mean   :-0.1662   Mean   :-0.2207  
 3rd Qu.: 0.0000   3rd Qu.: 0.0000   3rd Qu.: 0.0000   3rd Qu.: 0.0000  
 Max.   : 8.0000   Max.   : 8.0000   Max.   : 8.0000   Max.   : 8.0000  
      X10               X11       

In [7]:
credit_card <- as.data.frame(credit_card)
credit_card[,24] <- as.factor(credit_card[,24])

In [8]:
summary(credit_card)

       X1          X2        X3        X4              X5       
 Min.   :  10000   1:11888   1:10599   1:13713   Min.   :21.00  
 1st Qu.:  50000   2:18112   2:14030   2:15964   1st Qu.:28.00  
 Median : 140000             3: 4917   3:  323   Median :34.00  
 Mean   : 167484             4:  454             Mean   :35.49  
 3rd Qu.: 240000                                 3rd Qu.:41.00  
 Max.   :1000000                                 Max.   :79.00  
       X6                X7                X8                X9         
 Min.   :-2.0000   Min.   :-2.0000   Min.   :-2.0000   Min.   :-2.0000  
 1st Qu.:-1.0000   1st Qu.:-1.0000   1st Qu.:-1.0000   1st Qu.:-1.0000  
 Median : 0.0000   Median : 0.0000   Median : 0.0000   Median : 0.0000  
 Mean   :-0.0167   Mean   :-0.1338   Mean   :-0.1662   Mean   :-0.2207  
 3rd Qu.: 0.0000   3rd Qu.: 0.0000   3rd Qu.: 0.0000   3rd Qu.: 0.0000  
 Max.   : 8.0000   Max.   : 8.0000   Max.   : 8.0000   Max.   : 8.0000  
      X10               X11       

In [9]:
library("randomForest")
rf.model <- randomForest(Y~., data = credit_card, mtry = 23,importance = TRUE)

Warning message:
"package 'randomForest' was built under R version 3.3.3"randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.


In [10]:
summary(rf.model)

                Length Class  Mode     
call                5  -none- call     
type                1  -none- character
predicted       30000  factor numeric  
err.rate         1500  -none- numeric  
confusion           6  -none- numeric  
votes           60000  matrix numeric  
oob.times       30000  -none- numeric  
classes             2  -none- character
importance         92  -none- numeric  
importanceSD       69  -none- numeric  
localImportance     0  -none- NULL     
proximity           0  -none- NULL     
ntree               1  -none- numeric  
mtry                1  -none- numeric  
forest             14  -none- list     
y               30000  factor numeric  
test                0  -none- NULL     
inbag               0  -none- NULL     
terms               3  terms  call     

In [10]:
rf.model.pred <- predict(rf.model, credit_card, type = "class")

In [11]:
table(rf.model.pred, credit_card$Y)

             
rf.model.pred     0     1
            0 23358    15
            1     6  6621

In [12]:
(23356+6623)/(23356+6623+8+13)

[1] 0.9993

#### Now we will try 10 predictors

In [13]:
rf.model.few <- randomForest(Y~., data = credit_card, mtry = 10,importance = TRUE)

In [14]:
rf.model.few.pred <- predict(rf.model.few, credit_card, type = "class")

In [15]:
table(rf.model.few.pred, credit_card$Y)

                 
rf.model.few.pred     0     1
                0 23357    30
                1     7  6606

#### Now we will divide into train and test

In [11]:
train <- sample(30000, 24000)
rf.model <- randomForest(Y~., data = credit_card, subset = train, mtry = 23, importance = TRUE)
rf.model.pred <- predict(rf.model, credit_card[-train,], type = "class")
table(rf.model.pred, credit_card[-train,]$Y)

             
rf.model.pred    0    1
            0 4372  823
            1  295  510

In [17]:
rf.model <- randomForest(Y~., data = credit_card, subset = train, mtry = 10, importance = TRUE)
rf.model.pred <- predict(rf.model, credit_card[-train,], type = "class")
table(rf.model.pred, credit_card[-train,]$Y)

             
rf.model.pred    0    1
            0 4400  833
            1  266  501

In [18]:
(4377+506)/(4377+506+287+830)

[1] 0.8138333

In [19]:
importance(rf.model)

,0,1,MeanDecreaseAccuracy,MeanDecreaseGini
X1,25.196482,19.0727467,32.192655,473.63280
X2,3.954007,1.7579449,4.469540,67.86227
X3,3.059373,0.1949260,2.699114,147.59947
X4,13.996367,-0.9942568,11.848815,80.61739
X5,25.189381,3.5382108,24.646319,572.49991
X6,104.808153,103.1193337,170.545753,959.66617
X7,38.862818,7.6578077,44.953124,392.51770
X8,32.486858,6.3657383,37.066539,215.97826
X9,35.134408,2.6874633,39.063918,150.17815
X10,25.513833,8.3707234,32.677213,120.96218


### Now we will use SVM

In [12]:
library(e1071)
svm.model <- svm(Y~., data = credit_card[train,], kernel = "linear", cost = 1, scala = FALSE)

Warning message:
"package 'e1071' was built under R version 3.3.3"

In [13]:
svm.model.pred <- predict(svm.model, credit_card[-train,])

In [14]:
table(svm.model.pred, credit_card[-train,]$Y)

              
svm.model.pred    0    1
             0 4537  983
             1  130  350

In [23]:
(4519+340)/(4519+996+145+340)

[1] 0.8098333

In [24]:
mean(svm.model.pred == credit_card[-train,]$Y)

[1] 0.8116667

In [25]:
svm.model <- svm(Y~., data = credit_card[train,], kernel = "linear", cost = 1, scala = TRUE)

In [26]:
svm.model.pred <- predict(svm.model, credit_card[-train,])

In [27]:
table(svm.model.pred, credit_card[-train,]$Y)

              
svm.model.pred    0    1
             0 4546 1010
             1  120  324

In [28]:
mean(svm.model.pred == credit_card[-train,]$Y)

[1] 0.8116667

#### No change when scaling is done.

In [15]:
svm.radial = svm(Y~., data = credit_card[train,], kernel = "radial", gamma = 1, cost = 1)

In [16]:
svm.model.pred <- predict(svm.radial, credit_card[-train,])

In [17]:
table(svm.model.pred, credit_card[-train,]$Y)

              
svm.model.pred    0    1
             0 4522 1079
             1  145  254

In [18]:
(4522+254) / (6000)

[1] 0.796